In [26]:
# Import Dependencies
import requests
from bs4 import BeautifulSoup
import pandas as pd
import random
import time

In [27]:
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)",
    "Accept-Language": "en-US,en;q=0.9"
}


In [28]:
BASE_URL = "https://www.daraz.com.np/catalog/?q=washing%20machine"

In [50]:
# Core Logic
def scrape_daraz_washing_machines(pages):
    products=[]

    for page in range(1, pages+1):
        print(f"Scraping page {page}...")
        # url = BASE_URL.format(page)
        url = "https://www.daraz.com.np/catalog/?q=washing%20machine"
        response = requests.get(url, headers= HEADERS)
        print(response.status_code)
        print(response.text[:1000])
        soup = BeautifulSoup(response.text, "lxml")
        print(soup.prettify()[:5000])

        # items = soup.find_all("div", class_="ant-col ant-col-20 ant-col-push-4 Jv5R8 css-1bkhbmc app")
        items = soup.select("div[data-qa-locator='product-item'], div[data-qa-locator='product-card']")
        print("Items found:", len(items))

        for item in items:
            #Name
            try: 
                anchor = item.find("div", class_="RfADt").find("a")
                name =  anchor.get("title", "").strip()
                
            except:
                name = None

            # Price
            try:
                price = item.find("span", class_="ooOxS").text.strip()
            except:
                price = None

            # Rating
            try:
                rating = item.find("span", class_="qzqFw").text.strip()
            except:
                rating = None

            # Location
            try:
                location = item.find("span", class_="oa6ri").text.strip()
            except:
                rating = None

            # Reviews count
            try:
                reviews = item.find("span", class_="Wphh3").text.strip("()")
            except:
                reviews = None

            # Discount / Offer
            try:
                offer = item.find("span", class_="IcOsH").text.strip()
            except:
                offer = None

            # Extra offers / tags (like “Free Delivery”, “Mall”)
            try:
                extra_offers = ", ".join(
                    tag.text.strip()
                    for tag in item.find_all("span", class_="oa6ri")
                )
            except:
                extra_offers = None

            # Product URL
            try:
                link = "https://www.daraz.pk" + item.find("a")["href"]
            except:
                link = None

            products.append({
                "name": name,
                "price": price,
                "rating": rating,
                "reviews": reviews,
                "offer": offer,
                "extra_offers": extra_offers,
                "url": link
            })

        time.sleep(random.uniform(1.5,3))

    return products

In [51]:
# Run Scraper
data = scrape_daraz_washing_machines(pages=2)

Scraping page 1...
200

<!DOCTYPE html>
<html lang="en">
  <head>
    <meta charset="utf-8" />
    <meta name="data-spm" content="a2a0e" />
    <meta http-equiv="x-ua-compatible" content="ie=edge" />
    <meta name="viewport" content="width=device-width" />
    <meta name="referrer" content="origin" />
    <meta name="lzd-itrace-close-auto-monitor" content="assetPerf,imagesPerf">
    
      <meta name="aplus-rhost-v" content="sg.mmstat.com">
      <meta name="aplus-rhost-g" content="sg.mmstat.com">
      <meta name="aplus-cpvdata" content="{&quot;daraz_web_version&quot;:&quot;2.0.0&quot;}">
      <meta name="aplus-exdata" content="{&quot;daraz_web_version&quot;:&quot;2.0.0&quot;}">
    
    <link rel="preconnect dns-prefetch" href="//cart.daraz.com.np" />
    <link rel="preconnect dns-prefetch" href="//member.daraz.com.np" />
    <link rel="preconnect dns-prefetch" href="//acs-m.daraz.com.np" />
    <link rel="preconnect dns-prefetch" href="//laz-img-cdn.alicdn.com" />
    <link rel="p

In [35]:
# Convert to DataFrame and save CSV
df = pd.DataFrame(data)
df.drop_duplicates(inplace = True)

df.to_csv("daraz_washing_machines.csv", index =False)
df.head()

""
